In [3]:
import os
import numpy as np
#import scipy.sparse as sp
import pandas as pd
#import gc
import re, unicodedata
#from collections import defaultdict
import string
#import tqdm


# this will not be used,
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from sklearn.naive_bayes import MultinomialNB, BernoulliNB

In [7]:
!pwd

/Users/Matteo/projects/fundamentals-of-ml/kaggle/arxiv_classification/notebooks


In [8]:
def load_data():
    train = pd.read_csv("~/projects/fundamentals-of-ml/kaggle/arxiv_classification/data/train.csv")
    test = pd.read_csv("~/projects/fundamentals-of-ml/kaggle/arxiv_classification/data/test.csv")
    return train, test

In [9]:
def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return ''.join(new_words)

def process(df, t):
    df[t] = df[t].apply(lambda x : x.lower())
    #train['Abstract'] = train['Abstract'].apply(lambda x : remove_punctuation(x))
    df[t] = df[t].apply(lambda x : x.strip())
    df[t] = df[t].apply(lambda x : re.sub('\n', ' ', x))
    df[t] = df[t].apply(lambda x : re.sub('\[[^]]*\]', '', x))
    df[t] = df[t].apply(lambda x : re.sub("<.*?>", " ", x))
    df[t] = df[t].apply(lambda x : remove_non_ascii(x))
    print(df.head())
    return df

In [10]:
train, test = load_data()
train = process(train, 'Abstract')
test = process(test, 'Abstract')

   Id                                           Abstract           Category
0   0  the energy released in a solar flare is partit...           astro-ph
1   1  in light of current atmospheric neutrino oscil...             hep-ph
2   2  we consider the following basic learning task:...              cs.LG
3   3  in this paper, we characterise the family of f...            math.CO
4   4  the control of condensed matter systems out of...  cond-mat.mes-hall
   Id                                           Abstract
0   0  we describe ways to define and calculate $l_1$...
1   1  the progenitor systems of type-ia supernovae (...
2   2  omegawhite is a wide-field, high cadence, syno...
3   3  given $n \geq 2$ and $1<p<n$, we consider the ...
4   4  the motivation of this work is to improve the ...


In [15]:
class BernoulliVectorizer:
    
    def __init__(self):
        self.vocab = []
        self.vocab_counter = {}
        
    def build_vocab(self, data):
        for document in data:
            for word in document.split(' '):
                if word in self.vocab:
                    self.vocab_counter[str(word)] += 1
                else:
                    self.vocab.append(str(word))
                    self.vocab_counter = 1
                
    def transform(self, data):
        '''  this is some text" -----> (0, 0, 0, 0, 0, 1 ...) '''
        i = 0
        for document in data:
            tokens = document.split(' ') # ['my', 'name', 'is', 'kasper', 'and', 'i', 'like', 'ml']
                                         # ['my', 'kasper', 'hello', ..................]
                                         # (1, 1, 0, 0, )
            bin_vect = np.zeros(len(self.vocab))
            for word_idx in range(len(self.vocab)):
                for e in tokens: 
                    if e == self.vocab[word_idx]:
                        bin_vect[word_idx] = 1
                data[i] = bin_vect   # please do something else AAAAHH
                i += 1
                
        return data
    
    def fit_transform(self, data):
        self.build_vocab(data)
        return transform(data)

In [16]:
B = BernoulliVectorizer()
train['Abstract'] = B.fit_transform(train['Abstract'])
test['Abstract'] = B.transform(test['Abstract'])        # transform only

TypeError: 'int' object is not subscriptable

In [ ]:
train.head() # (0,0,0,0,1,0,0), (0,0,0,0,1,0,0), (0,0,0,0,1,0,0), (0,0,0,0,1,0,0), (0,0,0,0,1,0,0), 

In [ ]:
train, test = train.to_numpy(), test.to_numpy()

In [ ]:
class BernoulliNB:
    def __init__(self, alpha):
        self.alpha = alpha
        pass
    
    '''
    
    P(C_k) = number of documents of that class / number of documents
    P(C_k | w1, w2 ,... ) \prop P(C_k) P(w1| C_k) P(w2 | C_k) ..
    P(wi | C_k) = number of documents of class C_k with wi / number of documents with that class
     get all the rows of class C_k, how many of them has word wi
     get all the rows of class C_k, how many of them has a 1 in the index of wi
    '''
    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        n_classes = self.n_classes
        
        # calculate P(C_k) for all k
        self.counts = np.zeros(n_classes)
        for i in y:
            self.counts[i] += 1
        self.counts /= len(y)
        
        
        # generate n_features x n_classes matrix
        self.params = np.zeros((n_classes, X.shape[1]))
        for idx in range(len(X)):
            self.params[y[idx]] += X[idx]
        self.params += self.alpha #1 # Laplace
        class_sums = self.params.sum(axis=1) + self.alpha * self.n_classes # Laplace
        self.params = self.params / class_sums[:, np.newaxis]
        
        
    def predict(self, X):
        neg_prob = np.log(1 - self.params)
        # Compute  neg_prob · (1 - X).T  as  ∑neg_prob - X · neg_prob
        safe_sparse_dot
        jll = np.dot(X, (np.log(self.params) - neg_prob).T)
        
        # posterior calc where np.log(self.counts)=prior, neg_prob.sum(axis=1)=likelihood
        jll += np.log(self.counts) + neg_prob.sum(axis=1)
        
        # return max of posterior
        return np.argmax(jll, axis=1)
    